**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

C:\env\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The agent has to exploit what it has already experienced in order to obtain reward, but it also has to explore in order to make better action selections in the future. The dilemma is that neither exploration nor exploitation can be pursued exclusively without failing at the task. The agent must try a variety of actions and progressively favor those that appear to be best.

The $\epsilon$ made the agent continued to explore and to improve their chances of recognizing the optimal action.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        if np.count_nonzero(self.board == 0.5) == 0:
          reward += 5
          game_over = True
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [2]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=101 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The position and board embed the pairs of coordinates to a fixed size of array, shape of (5,5,2) for example. In this way, the agent can capture more information from the environment, more importantly, the agent can use a CNN network to capture the information of channels in the environment. And the gradient will converge faster.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        
        return np.random.choice([0,1,2,3])

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [18]:
def test(agent,env,epochs,prefix=''):
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        
        while not game_over:
        # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over ,com= env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Save as a mp4
        env.draw(prefix+str(e))

            # Update stats
        score = score + win-lose
            
        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('\nFinal score: '+str(score/epochs))
    

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,10,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.0/16.0. Average score (-5.0)
Win/lose count 8.5/22.0. Average score (-9.25)
Win/lose count 9.0/12.0. Average score (-7.166666666666667)
Win/lose count 14.0/14.0. Average score (-5.375)
Win/lose count 9.0/11.0. Average score (-4.7)
Win/lose count 4.5/7.0. Average score (-4.333333333333333)
Win/lose count 9.0/15.0. Average score (-4.571428571428571)
Win/lose count 8.0/15.0. Average score (-4.875)
Win/lose count 8.0/12.0. Average score (-4.777777777777778)
Win/lose count 5.5/6.0. Average score (-4.35)

Final score: -4.35


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__1__
\begin{split}
Q^\pi (s,a) &= \mathbb{E}[R_t + G_{t+1}]\\
&\doteq \mathbb{E} [R_{t+1} + \gamma v_{\pi}(S_{t+1})\ |\ S_t = s, A_t = a]\\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{split}

__2__
\begin{split}
Q^\pi (s,a) &= max(\mathbb{E}[R_t + G_{t+1}])\\
&\doteq \mathbb{E} [R_{t} + max(\gamma v_{\pi}(S_{t+1})\ |\ S_t = s, A_t = a)]\\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma max_{a'} Q^{*}(s',a')]
\end{split}

__3__

Optimal Q:
$$Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].$$
Q from agent:
$$Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$$

from above, it is obvious to get the loss function:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
from collections import deque
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = deque(maxlen = self.max_memory)

    def remember(self, m):
        self.memory.append(m)
        

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]


***
The pipeline we will use for training is given below:

In [15]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    agent.epsilon = 0.9
    for e in range(epoch):
        agent.epsilon = agent.epsilon * 0.9
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over,com = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 2 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
#         print(agent.epsilon)
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))

        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        aa = np.argmax(self.model.predict(s.reshape(1,5,5,self.n_state)))
        return aa


    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state)) #16,5,5,2
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s, n_s, a, r, game_over = self.memory.random_access()
            
            if game_over_:
                target_q[i][a] = r
            else:
                input_states[i] = s
                q = self.model.predict(n_s[np.newaxis,:])
                target_q[i][a] = r + self.discount * np.max(q)

        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)

        return l
    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Dense(units=32,input_shape = (5,5,self.n_state),activation="relu"))
        model.add(keras.layers.Flatten())
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(4,activation="linear"))        
        model.compile(sgd(lr=lr, decay=1e-2, momentum=0.0), "mse")
        self.model = model
        

In [123]:
env = Environment(grid_size=size, max_time=256, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 30, prefix='fc_train')
HTML(display_videos('fc_train20.mp4'))

Epoch 000/030 | Loss 0.0277 | Win/lose count 11.5/15.0 (-3.5)
Epoch 001/030 | Loss 0.0162 | Win/lose count 13.5/12.0 (1.5)
Epoch 002/030 | Loss 0.0091 | Win/lose count 16.5/12.0 (4.5)
Epoch 003/030 | Loss 0.0210 | Win/lose count 10.0/15.0 (-5.0)
Epoch 004/030 | Loss 0.0103 | Win/lose count 14.5/23.0 (-8.5)
Epoch 005/030 | Loss 0.0250 | Win/lose count 12.0/13.0 (-1.0)
Epoch 006/030 | Loss 0.0146 | Win/lose count 9.5/12.0 (-2.5)
Epoch 007/030 | Loss 0.0042 | Win/lose count 7.5/14.0 (-6.5)
Epoch 008/030 | Loss 0.0181 | Win/lose count 9.0/19.0 (-10.0)
Epoch 009/030 | Loss 0.0284 | Win/lose count 9.5/4.0 (5.5)
Epoch 010/030 | Loss 0.0181 | Win/lose count 10.0/19.0 (-9.0)
Epoch 011/030 | Loss 0.0182 | Win/lose count 14.0/10.0 (4.0)
Epoch 012/030 | Loss 0.0232 | Win/lose count 7.0/21.0 (-14.0)
Epoch 013/030 | Loss 0.0153 | Win/lose count 6.5/9.0 (-2.5)
Epoch 014/030 | Loss 0.0405 | Win/lose count 9.5/5.0 (4.5)
Epoch 015/030 | Loss 0.0166 | Win/lose count 8.5/7.0 (1.5)
Epoch 016/030 | Loss 0.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

# CNN


In [12]:

class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        input_shape = (5,5, self.n_state)
        model.add(Conv2D(64,(2,2), input_shape=input_shape))
        model.add(Conv2D(32, (2,2)))
        model.add(Flatten())
        model.add(Dense(16, activation='relu'))
        model.add(Dense(4, activation='linear'))

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [128]:
from keras.layers import Flatten
env = Environment(grid_size=size, max_time=256, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.2, memory_size=2000, batch_size = 32)
train(agent,env,101,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/101 | Loss 0.0468 | Win/lose count 16.0/29.0 (-13.0)
Epoch 001/101 | Loss 0.0030 | Win/lose count 10.0/11.0 (-1.0)
Epoch 002/101 | Loss 0.0200 | Win/lose count 9.5/15.0 (-5.5)
Epoch 003/101 | Loss 0.0291 | Win/lose count 7.0/8.0 (-1.0)
Epoch 004/101 | Loss 0.0262 | Win/lose count 19.0/18.0 (1.0)
Epoch 005/101 | Loss 0.0146 | Win/lose count 13.0/19.0 (-6.0)
Epoch 006/101 | Loss 0.0220 | Win/lose count 8.5/7.0 (1.5)
Epoch 007/101 | Loss 0.0262 | Win/lose count 21.5/13.0 (8.5)
Epoch 008/101 | Loss 0.0146 | Win/lose count 4.5/6.0 (-1.5)
Epoch 009/101 | Loss 0.0027 | Win/lose count 9.5/13.0 (-3.5)
Epoch 010/101 | Loss 0.0024 | Win/lose count 6.5/12.0 (-5.5)
Epoch 011/101 | Loss 0.0218 | Win/lose count 11.0/4.0 (7.0)
Epoch 012/101 | Loss 0.0106 | Win/lose count 11.5/5.0 (6.5)
Epoch 013/101 | Loss 0.0029 | Win/lose count 15.5/7.0 (8.5)
Epoch 014/101 | Loss 0.0124 | Win/lose count 11.5/3.0 (8.5)
Epoch 015/101 | Loss 0.0025 | Win/lose count 3.5/2.0 (1.5)
Epoch 016/101 | Loss 0.0127 | 

KeyboardInterrupt: 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

the agent sometimes just trapped in a corner and cannot explore the whole map.

In [130]:
env = Environment(grid_size=size, max_time=256,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 43.0/1.0. Average score (42.0)
Win/lose count 43.5/4.0. Average score (40.75)
Win/lose count 19.5/2.0. Average score (33.0)
Win/lose count 6.0/0. Average score (26.25)
Win/lose count 30.0/4.0. Average score (26.2)
Win/lose count 8.5/0. Average score (23.25)
Win/lose count 8.5/2.0. Average score (20.857142857142858)
Win/lose count 23.5/3.0. Average score (20.8125)
Win/lose count 8.0/2.0. Average score (19.166666666666668)
Win/lose count 13.0/1.0. Average score (18.45)
Win/lose count 22.5/1.0. Average score (18.727272727272727)

Final score: 18.727272727272727
Test of the FC
Win/lose count 1.0/1.0. Average score (0.0)
Win/lose count 2.0/2.0. Average score (0.0)
Win/lose count 0.5/0. Average score (0.16666666666666666)
Win/lose count 1.5/0. Average score (0.5)
Win/lose count 1.5/1.0. Average score (0.5)
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 2.5/0. Average score (0.7857142857142857)
Win/lose count 1.5/0. Average score (0.875)
Win/lose count

In [132]:
HTML(display_videos('cnn_test0.mp4'))

In [133]:
HTML(display_videos('fc_test0.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [13]:
def train_explore(agent,env,epoch,prefix=''):
    
    # Number of won games
    score = 0
    loss = 0
    train = True
    agent.epsilon = 0.9
    for e in range(epoch):
        agent.epsilon = agent.epsilon * 0.9
        state = env.reset()
        game_over = False

        win = 0
        lose = 0
        com= False
        while not game_over:
            action = agent.act(state)
            prev_state = state
            state, reward, game_over,com = env.act(action, train)

              # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

              # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

          # Save as a mp4
            
            if e % 10 == 0:
                env.draw(prefix+str(e))
            if com:
                env.draw(prefix+str(e))

          # Update stats
            score += win-lose
            
        print("\nEpoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.1f}/{:.1f} ({:.1f})"
                .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    
            
        
class EnvironmentExploring():
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature        
        
        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16        
        
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
        
    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b

    def act(self, action, train=False):
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))
        
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        game_over = self.t > self.max_time
        com = False
        reward = 0
        if train: 
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1
        
        reward = reward + self.board[self.x, self.y]
        if np.count_nonzero(self.board == 0.5) == 0:
            game_over = True
            reward += 30
            com = True
            print('eat it up')
        self.board[self.x, self.y] = 0
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state, reward, game_over,com
    
    def reset(self):
        """This function resets the game and returns the initial state"""
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        
        self.board[0:2,:]= 0
        self.board[:,0:2] = 0
        self.board[-2:, :] = 0
        self.board[:, -2:] = 0
        
        self.board[self.x,self.y] = 0
        self.t = 0
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [24]:
from keras.layers import Flatten
# Training

env = EnvironmentExploring(grid_size=size, max_time=300, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.01, memory_size=2500, batch_size = 32,n_state=3)
train_explore(agent, env, 128, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))
HTML(display_videos('random1.mp4'))


Epoch 000/128 | Loss 0.0448 | Win/lose count 8.5/98.7 (-90.2)

Epoch 001/128 | Loss 0.0538 | Win/lose count 15.0/74.0 (-59.0)

Epoch 002/128 | Loss 0.0429 | Win/lose count 18.0/64.3 (-46.3)

Epoch 003/128 | Loss 0.0429 | Win/lose count 16.0/76.1 (-60.1)

Epoch 004/128 | Loss 0.0437 | Win/lose count 21.0/54.2 (-33.2)

Epoch 005/128 | Loss 0.0743 | Win/lose count 24.0/43.8 (-19.8)

Epoch 006/128 | Loss 0.0605 | Win/lose count 23.5/49.3 (-25.8)

Epoch 007/128 | Loss 0.0491 | Win/lose count 20.0/53.1 (-33.1)

Epoch 008/128 | Loss 0.0448 | Win/lose count 17.5/66.4 (-48.9)

Epoch 009/128 | Loss 0.0362 | Win/lose count 18.5/56.0 (-37.5)

Epoch 010/128 | Loss 0.0468 | Win/lose count 21.0/46.8 (-25.8)

Epoch 011/128 | Loss 0.0252 | Win/lose count 25.5/35.1 (-9.6)

Epoch 012/128 | Loss 0.0356 | Win/lose count 27.5/34.6 (-7.1)

Epoch 013/128 | Loss 0.0586 | Win/lose count 20.5/48.6 (-28.1)

Epoch 014/128 | Loss 0.0614 | Win/lose count 26.0/39.4 (-13.4)

Epoch 015/128 | Loss 0.0358 | Win/lose cou

KeyboardInterrupt: 

In [22]:
from keras.layers import Flatten
env = EnvironmentExploring(grid_size=size, max_time=100, temperature=0.3)
agent1 = DQN_CNN(size, lr=.1, epsilon = 0.01, memory_size=2500, batch_size = 64,n_state=3)
agent1.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')
test(agent1,env,11,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 18.5/5.0. Average score (13.5)
Win/lose count 17.5/5.0. Average score (13.0)
Win/lose count 17.5/3.0. Average score (13.5)
Win/lose count 18.0/2.0. Average score (14.125)
Win/lose count 20.5/1.0. Average score (15.2)
Win/lose count 16.0/2.0. Average score (15.0)
Win/lose count 21.5/6.0. Average score (15.071428571428571)
Win/lose count 18.5/1.0. Average score (15.375)
Win/lose count 21.5/2.0. Average score (15.833333333333334)
Win/lose count 20.0/5.0. Average score (15.75)
Win/lose count 19.5/3.0. Average score (15.818181818181818)

Final score: 15.818181818181818


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

I mortified the enviornment such that if all the cheese are eated the game is over and give it a big reward. Meanwhile, I make the max time longer to make the agent to achieve the goal

***

In [18]:
class EnvironmentExploring2():
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature        
        
        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16        
        
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
        pass
        
    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b

    def act(self, action, train=False):
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))
        
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        game_over = self.t > self.max_time
        reward = 0
        if train: 
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = self.malus_position[self.x, self.y]*1.001
        self.malus_position[self.x, self.y] += 0.1
        
        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        if np.count_nonzero(self.board == 0.5) == 0:
            game_over = True
            reward += 30
            print('completed')
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state, reward, game_over    
    
    def reset(self):
        """This function resets the game and returns the initial state"""
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        
        self.board[0:2,:]= 0
        self.board[:,0:2] = 0
        self.board[-2:, :] = 0
        self.board[:, -2:] = 0
        self.board[self.x,self.y] = 0
        self.t = 0
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [26]:
from keras.layers import Flatten
env2 = EnvironmentExploring2(grid_size=size, max_time=512, temperature=0.5)
agent2 = DQN_CNN(size, lr=.1, epsilon = 0.01, memory_size=2500, batch_size = 64,n_state=3)
train_explore(agent2, env2, 128, prefix='cnn2_train_explore')




Epoch 000/128 | Loss 0.0790 | Win/lose count 29.0/173.7 (-144.7)

Epoch 001/128 | Loss 0.0784 | Win/lose count 30.0/182.4 (-152.4)

Epoch 002/128 | Loss 0.0449 | Win/lose count 37.5/135.1 (-97.6)

Epoch 003/128 | Loss 0.0561 | Win/lose count 28.0/150.4 (-122.4)

Epoch 004/128 | Loss 0.0461 | Win/lose count 36.5/142.9 (-106.4)

Epoch 005/128 | Loss 0.0584 | Win/lose count 39.5/109.4 (-69.9)

Epoch 006/128 | Loss 0.0487 | Win/lose count 44.0/132.7 (-88.7)

Epoch 007/128 | Loss 0.0605 | Win/lose count 39.5/118.7 (-79.2)

Epoch 008/128 | Loss 0.0452 | Win/lose count 38.5/116.0 (-77.5)

Epoch 009/128 | Loss 0.0444 | Win/lose count 43.5/104.4 (-60.9)

Epoch 010/128 | Loss 0.0428 | Win/lose count 45.5/102.1 (-56.6)

Epoch 011/128 | Loss 0.0547 | Win/lose count 36.5/133.1 (-96.6)

Epoch 012/128 | Loss 0.0730 | Win/lose count 38.5/146.1 (-107.6)

Epoch 013/128 | Loss 0.0526 | Win/lose count 37.0/136.6 (-99.6)

Epoch 014/128 | Loss 0.0549 | Win/lose count 39.0/156.5 (-117.5)

Epoch 015/128 | Lo

KeyboardInterrupt: 

In [31]:
# from keras.layers import Flatten
env3 = EnvironmentExploring2(grid_size=size, max_time=1024, temperature=0.5)
# agent1 = DQN_CNN(size, lr=.1, epsilon = 0.01, memory_size=2500, batch_size = 64,n_state=3)
# agent1.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')
test(agent,env3,11,prefix='cnn_test_explore')
# HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 38.0/15.0. Average score (23.0)
Win/lose count 31.0/10.0. Average score (22.0)
Win/lose count 34.0/25.0. Average score (17.666666666666668)
Win/lose count 46.0/11.0. Average score (22.0)
Win/lose count 38.0/28.0. Average score (19.6)
Win/lose count 44.5/31.0. Average score (18.583333333333332)
Win/lose count 35.0/20.0. Average score (18.071428571428573)
Win/lose count 37.0/18.0. Average score (18.1875)
Win/lose count 39.0/17.0. Average score (18.61111111111111)
Win/lose count 41.0/18.0. Average score (19.05)
Win/lose count 38.5/16.0. Average score (19.363636363636363)

Final score: 19.363636363636363


In [33]:
def test1(agent,env,epochs,prefix=''):
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        
        while not game_over:
        # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over,com = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            # Save as a mp4
        env.draw(prefix+str(e))

            # Update stats
        score = score + win-lose
            
        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('\nFinal score: '+str(score/epochs))
    

In [36]:
env3 = EnvironmentExploring(grid_size=13, max_time=360, temperature=0.2)
agent1 = DQN_CNN(size, lr=.1, epsilon = 0.01, memory_size=2500, batch_size = 64,n_state=3)
agent1.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')
test1(agent1,env3,12,prefix='cnn_test_explorexx')

eat it up
Win/lose count 43.5/10.0. Average score (33.5)
Win/lose count 15.5/10.0. Average score (19.5)
Win/lose count 13.0/5.0. Average score (15.666666666666666)
Win/lose count 12.5/11.0. Average score (12.125)
Win/lose count 16.0/7.0. Average score (11.5)
Win/lose count 11.0/11.0. Average score (9.583333333333334)
Win/lose count 13.5/6.0. Average score (9.285714285714286)
Win/lose count 10.5/14.0. Average score (7.6875)
Win/lose count 13.0/3.0. Average score (7.944444444444445)
Win/lose count 19.0/6.0. Average score (8.45)
Win/lose count 16.0/7.0. Average score (8.5)
Win/lose count 15.5/8.0. Average score (8.416666666666666)

Final score: 8.416666666666666


In [37]:
HTML(display_videos('cnn_test_explorexx0.mp4'))

In [3]:
HTML(display_videos('cnn_test_explorexx2.mp4'))